# Intro to Recommender Systems Lab

Complete the exercises below to solidify your knowledge and understanding of recommender systems.

For this lab, we are going to be putting together a user similarity based recommender system in a step-by-step fashion. Our data set contains customer grocery purchases, and we will use similar purchase behavior to inform our recommender system. Our recommender system will generate 5 recommendations for each customer based on the purchases they have made.

In [1]:
import pandas as pd
from scipy.spatial.distance import pdist, squareform

In [2]:
data = pd.read_csv('../data/customer_product_sales.csv')

In [3]:
data.head()

,CustomerID,FirstName,LastName,SalesID,ProductID,ProductName,Quantity
0,61288,Rosa,Andersen,134196,229,Bread - Hot Dog Buns,16
1,77352,Myron,Murray,6167892,229,Bread - Hot Dog Buns,20
2,40094,Susan,Stevenson,5970885,229,Bread - Hot Dog Buns,11
3,23548,Tricia,Vincent,6426954,229,Bread - Hot Dog Buns,6
4,78981,Scott,Burch,819094,229,Bread - Hot Dog Buns,20


## Step 1: Create a data frame that contains the total quantity of each product purchased by each customer.

You will need to group by CustomerID and ProductName and then sum the Quantity field.

In [9]:
group = data.groupby(["CustomerID","ProductName"], as_index = False).agg({"Quantity": "sum"})
group.head()

,CustomerID,ProductName,Quantity
0,33,Apricots - Dried,1
1,33,Assorted Desserts,1
2,33,Bandage - Flexible Neon,1
3,33,"Bar Mix - Pina Colada, 355 Ml",1
4,33,"Beans - Kidney, Canned",1


## Step 2: Use the `pivot_table` method to create a product by customer matrix.

The rows of the matrix should represent the products, the columns should represent the customers, and the values should be the quantities of each product purchased by each customer. You will also need to replace nulls with zeros, which you can do using the `fillna` method.

In [8]:
pivot = pd.pivot_table(data, values='Quantity', index='ProductName', columns='CustomerID',fill_value=0)
pivot.head()

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
ProductName,,,,,,,,,,,,,,,,,,,,,
Anchovy Paste - 56 G Tube,0,0,0,0,0,0,0,1,0,0,...,0,25,0,0,0,0,0,0,0,0
"Appetizer - Mini Egg Roll, Shrimp",0,0,0,0,0,0,0,0,0,0,...,25,25,0,0,0,0,0,0,0,0
Appetizer - Mushroom Tart,0,0,0,0,0,0,0,1,0,0,...,25,0,0,0,0,0,0,0,25,0
Appetizer - Sausage Rolls,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,25,25,25,0,25,0
Apricots - Dried,1,0,0,0,1,0,0,0,0,0,...,0,25,0,0,0,0,0,0,0,0


In [32]:
#***
pivot2 = pivot.copy()
pivot2["Diferencias"] = (pivot2[33]-pivot2[200])**2
sumita = pivot2["Diferencias"].sum()
varianza = pivot2["Diferencias"].var()
total = (sumita)**(1/2)
total

10.723805294763608

## Step 3: Create a customer similarity matrix using `squareform` and `pdist`. For the distance metric, choose "euclidean."

In [33]:
distancias = squareform(pdist(pivot.T, 'euclidean'))
distancias
#***Calcula la distancia euclidiana literalmente, entre cada columna, por ejemplo la distancia de la columna 33 con la misma 33 es 
# 0 y con la siguiente que es la 200 nos da una distancia de 10.72
similitud = 1/(1+distancias) 
similitudes = pd.DataFrame(similitud, index = pivot.columns, columns = pivot.columns)
similitudes

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
CustomerID,,,,,,,,,,,,,,,,,,,,,
33,1.000000,0.085297,0.093953,0.091747,0.087410,0.089695,0.085297,0.088913,0.088152,0.089695,...,0.004809,0.005108,0.004996,0.005421,0.004920,0.005023,0.004880,0.005026,0.004549,0.004883
200,0.085297,1.000000,0.085638,0.085297,0.080070,0.083020,0.084959,0.083651,0.085638,0.087047,...,0.004825,0.005121,0.005014,0.005448,0.004925,0.005032,0.004909,0.005042,0.004553,0.004879
264,0.093953,0.085638,1.000000,0.088152,0.089301,0.087047,0.085638,0.086333,0.087047,0.087047,...,0.004822,0.005115,0.004996,0.005441,0.004932,0.005055,0.004894,0.005042,0.004566,0.004883
356,0.091747,0.085297,0.088152,1.000000,0.085983,0.086688,0.085983,0.091325,0.085983,0.087410,...,0.004814,0.005111,0.004999,0.005437,0.004920,0.005036,0.004871,0.005042,0.004563,0.004886
412,0.087410,0.080070,0.089301,0.085983,1.000000,0.085638,0.085638,0.089301,0.084959,0.087779,...,0.004808,0.005131,0.004996,0.005441,0.004925,0.005042,0.004876,0.005039,0.004568,0.004903
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97928,0.005023,0.005032,0.005055,0.005036,0.005042,0.005042,0.005042,0.005039,0.005039,0.005033,...,0.003732,0.003945,0.003749,0.004045,0.003964,1.000000,0.003945,0.003945,0.003579,0.003608
98069,0.004880,0.004909,0.004894,0.004871,0.004876,0.004877,0.004882,0.004880,0.004886,0.004886,...,0.003638,0.003765,0.003716,0.003782,0.003888,0.003945,1.000000,0.003907,0.003608,0.003765
98159,0.005026,0.005042,0.005042,0.005042,0.005039,0.005039,0.005039,0.005033,0.005045,0.005036,...,0.003732,0.003984,0.003852,0.003888,0.003817,0.003945,0.003907,1.000000,0.003732,0.003732


## Step 4: Check your results by generating a list of the top 5 most similar customers for a specific CustomerID.

In [60]:
ordenacion = similitudes[200].sort_values(ascending=False)[1:]
top5 = pd.DataFrame(ordenacion[:5])
top5

,200
CustomerID,
1920,0.094414
1072,0.092610
3253,0.090094
3317,0.090094
3909,0.089695


## Step 5: From the data frame you created in Step 1, select the records for the list of similar CustomerIDs you obtained in Step 4.

In [50]:
lista_indices = list(top5.index)
records = group[group['CustomerID'].isin(lista_indices)].reset_index(drop=True)
records

,CustomerID,ProductName,Quantity
0,1072,"Appetizer - Mini Egg Roll, Shrimp",1
1,1072,Appetizer - Sausage Rolls,1
2,1072,"Beans - Kidney, Canned",1
3,1072,Beef - Montreal Smoked Brisket,1
4,1072,Beef - Striploin Aa,1
...,...,...,...
256,3909,Wine - Charddonnay Errazuriz,2
257,3909,Wine - Fume Blanc Fetzer,1
258,3909,"Wine - Magnotta, Merlot Sr Vqa",1
259,3909,Wine - Toasted Head,1


## Step 6: Aggregate those customer purchase records by ProductName, sum the Quantity field, and then rank them in descending order by quantity.

This will give you the total number of each product purchased by the 5 most similar customers to the customer you selected in order from most purchased to least.

In [51]:
agrupacion = records.groupby('ProductName',as_index=False).agg({'Quantity':'sum'}).sort_values('Quantity', ascending=False).reset_index(drop=True)
agrupacion

,ProductName,Quantity
0,Soup - Campbells Bean Medley,3
1,Lamb - Ground,3
2,Pomello,3
3,Bay Leaf,3
4,Pork - Kidney,3
...,...,...
212,Kiwi,1
213,Knife Plastic - White,1
214,"Lamb - Pieces, Diced",1
215,Lambcasing,1


## Step 7: Filter the list for products that the chosen customer has not yet purchased and then recommend the top 5 products with the highest quantities that are left.

- Merge the ranked products data frame with the customer product matrix on the ProductName field.
- Filter for records where the chosen customer has not purchased the product.
- Show the top 5 results.

In [52]:
filtro = agrupacion.merge(pivot[200],on='ProductName')
filtro = filtro[filtro[200] == 0]
filtro.sort_values("Quantity",ascending=False,inplace=True)
filtro

,ProductName,Quantity,200
0,Soup - Campbells Bean Medley,3,0
3,Bay Leaf,3,0
4,Pork - Kidney,3,0
5,Muffin - Carrot Individual Wrap,3,0
6,Scampi Tail,3,0
...,...,...,...
102,"Turkey - Whole, Fresh",1,0
101,"Veal - Brisket, Provimi,bnls",1,0
100,Wonton Wrappers,1,0
99,Wine - Wyndham Estate Bin 777,1,0


In [53]:
recomendaciones = list(filtro[:5]["ProductName"])
recomendaciones

['Soup - Campbells Bean Medley',
 'Bay Leaf',
 'Pork - Kidney',
 'Muffin - Carrot Individual Wrap',
 'Scampi Tail']

## Step 8: Now that we have generated product recommendations for a single user, put the pieces together and iterate over a list of all CustomerIDs.

- Create an empty dictionary that will hold the recommendations for all customers.
- Create a list of unique CustomerIDs to iterate over.
- Iterate over the customer list performing steps 4 through 7 for each and appending the results of each iteration to the dictionary you created.

In [74]:
recomendaciones = {}
clientes = list(data["CustomerID"].unique())
for i in clientes: 
    ordenacion = similitudes[i].sort_values(ascending=False)[1:6]
    lista_indices = list(ordenacion.index)
    records = group[group['CustomerID'].isin(lista_indices)].reset_index(drop=True)
    agrupacion = records.groupby('ProductName',as_index=False).agg({'Quantity':'sum'}).sort_values('Quantity', ascending=False).reset_index(drop=True)
    filtro = agrupacion.merge(pivot[i],on='ProductName')
    filtro = filtro[filtro[i] == 0]
    filtro.sort_values("Quantity",ascending=False,inplace=True)
    recomendaciones[i] = list(filtro[:5]["ProductName"])

In [75]:
recomendaciones[33]

['Chocolate - Dark',
 'Bay Leaf',
 'Oranges - Navel, 72',
 'Wine - Ej Gallo Sierra Valley',
 'Pecan Raisin - Tarts']

##  Step 9: Store the results in a Pandas data frame. The data frame should a column for Customer ID and then a column for each of the 5 product recommendations for each customer.

In [86]:
rec_cliente = pd.DataFrame(recomendaciones)
rec_final = rec_cliente.T
rec_final

,0,1,2,3,4
61288,"Mushrooms - Black, Dried","Wine - Magnotta, Merlot Sr Vqa",Chicken - Soup Base,Wine - Chardonnay South,Milk - 1%
77352,Guinea Fowl,Grenadine,Ecolab - Mikroklene 4/4 L,"Oranges - Navel, 72","Coconut - Shredded, Sweet"
40094,"Water - Mineral, Natural","Oregano - Dry, Rubbed",Quiche Assorted,Pasta - Orecchiette,Beef - Inside Round
23548,Wanton Wrap,Banana Turning,Flavouring - Orange,Beer - Original Organic Lager,Puree - Passion Fruit
78981,Lettuce - Frisee,Longos - Chicken Wings,Pop Shoppe Cream Soda,Sprouts - Alfalfa,Beef - Inside Round
...,...,...,...,...,...
49005,Olive - Spread Tapenade,"Veal - Inside, Choice",Mayonnaise - Individual Pkg,Cumin - Whole,Sardines
41286,Veal - Inside,Bread Foccacia Whole,"Veal - Inside, Choice",Wasabi Powder,Rabbit - Whole
85878,Cookie - Dough Variety,"Beef - Ground, Extra Lean, Fresh",Muffin - Carrot Individual Wrap,Bread - Rye,Cheese - Camembert
68506,Pasta - Cheese / Spinach Bauletti,"Doilies - 5, Paper",Black Currants,Tea - Decaf Lipton,Cocoa Butter


## Step 10: Change the distance metric used in Step 3 to something other than euclidean (correlation, cityblock, consine, jaccard, etc.). Regenerate the recommendations for all customers and note the differences.

In [113]:
distancias2 = squareform(pdist(pivot.T, 'correlation'))
similitud2 = 1/(1+distancias2) 
similitudes2 = pd.DataFrame(similitud2, index = pivot.columns, columns = pivot.columns)
recomendaciones2 = {}
clientes2 = list(data["CustomerID"].unique())


In [114]:
for i in clientes2: 
    ordenacion2 = similitudes2[i].sort_values(ascending=False)[1:6]
    lista_indices2 = list(ordenacion2.index)
    records2 = group[group['CustomerID'].isin(lista_indices2)].reset_index(drop=True)
    agrupacion2 = records2.groupby('ProductName',as_index=False).agg({'Quantity':'sum'}).sort_values('Quantity', ascending=False).reset_index(drop=True)
    filtro2 = agrupacion2.merge(pivot[i],on='ProductName')
    filtro2 = filtro2[filtro2[i] == 0]
    filtro2.sort_values("Quantity",ascending=False,inplace=True)
    recomendaciones2[i] = list(filtro2[:5]["ProductName"])

In [115]:
recomendaciones2[33]

['Rosemary - Primerba, Paste',
 'Cornflakes',
 'Beef - Short Loin',
 'Juice - Apple Cider',
 'Chinese Foods - Chicken']

In [116]:
recomendaciones[33]

['Chocolate - Dark',
 'Bay Leaf',
 'Oranges - Navel, 72',
 'Wine - Ej Gallo Sierra Valley',
 'Pecan Raisin - Tarts']